In [3]:
%pip install langchain_community openai elasticsearch tiktoken langchain_elasticsearch langchain_openai cohere python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Chatbot Implementation 

This section of the tutorial discusses the most interesting aspects of the chatbot implementation, to help you understand it and customize it.



## Ingest



In [4]:
class Search: 
    def __init__(self, es_client, model_id, index):
        self.es = es_client
        self.model_id = model_id
        self.index = index
        print('Created to Elasticsearch! successfully')

    def sparse_search_default(self, query, top_k=5): 
        search_query = {
            "bool": {
                "must": [
                    {
                        "text_expansion": {
                            f"text_embedding": {
                                "model_id": self.model_id,
                                "model_text": query,
                            }
                        }
                    }
                ]
            }
        }
        
        response = self.es.search(
            index=self.index, 
            query=search_query, 
            size=top_k
        )
        
        def extract_info(hit):
            source = hit["_source"]
            return {
                "id": hit["_id"],
                "index": hit["_index"],
                "score": hit["_score"],
                "column1": source.get("column1"),
                "column2": source.get("column2"),
                "model_id": source.get("model_id")
            }
        
        results = [extract_info(hit) for hit in response["hits"]["hits"]]

        return results


'35e14e0f2e12492e832ab6a800d0b2c1:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGNmYjVlMzZkMzhjMjRhODA4NTA4NzgyNDRmOTNjMjg0JGYyNDFiNzhiMDY5MjQ4NzFhY2RkZmFiOWMyMzg5MTdl'

In [3]:
from elasticsearch import Elasticsearch, NotFoundError
from langchain_elasticsearch import ElasticsearchStore
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_elasticsearch import DenseVectorStrategy
import os 
from dotenv import load_dotenv

load_dotenv(override=True)



INDEX = "chatbot-rag-v2"
ELASTICSEARCH_URL = os.getenv("ELASTICSEARCH_URL")
ELASTIC_CLOUD_ID = os.getenv("ELASTIC_CLOUD_ID")
ELASTIC_API_KEY = os.getenv("ELASTIC_API_KEY")
ELSER_MODEL = os.getenv("ELSER_MODEL")

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=512, chunk_overlap=256
)

# elasticsearch_client = Elasticsearch(
#     [ES_HOST],
#     basic_auth=(ES_USER, ES_PASSWORD),
#     ssl_context=ssl_context     
# )

elasticsearch_client = Elasticsearch(cloud_id=ELASTIC_CLOUD_ID, api_key=ELASTIC_API_KEY)

# load docs 
def load_docs(): 
    split_docs = [] 

    with open('data.json', 'rt') as f:
        documents = json.loads(f.read())
        docs = [Document(page_content=doc['content']) for doc in documents[:20]]
        split_docs = text_splitter.transform_documents(docs)

    return split_docs


# write documents stored in "docs" to the index
# langchain version of ES 
# store = ElasticsearchStore(
#     es_connection=elasticsearch_client,
#     index_name=INDEX,
#     strategy=ElasticsearchStore.SparseVectorRetrievalStrategy(model_id=ELSER_MODEL), 
# )


search = Search(elasticsearch_client, ELSER_MODEL, INDEX)

NameError: name 'Search' is not defined

In [6]:
question = "How to avoid muscle soreness after running?"

results = search.sparse_search_default(question, top_k=5)

[result["column2"] for result in results]

['The severity of soreness you experience depends on the type of stress placed on the muscle. Exercise with a strong eccentric component, for example, running downhill, is shown to increase the chance of DOMS, which can last up to 5 days post workout.OMS occurs within hours or days after your workout and often causes tenderness, swelling, and stiffness in the joints. Starting recovery immediately after your workout can greatly reduce muscle soreness and fatigue, and have you well on your way to a stronger, healthier body. Here is what we suggest. Water.',
 'While you can’t avoid muscle soreness completely as a runner, there are things you can do to lessen it. When you think of treatments for muscles recently beat up by the roads, you probably think of things such as ice baths, massage, topical analgesics and anti-inflammatory medications.',
 "Your muscles might feel sore after a run or overdoing it at the gym, particularly if you don't normally exercise as rigorously. This occurs due t

In [10]:
from langchain_core.prompts import ChatPromptTemplate

ANSWER_PROMPT_TEMPLATE = ChatPromptTemplate.from_template(
""" 
당신은 지식 기반 질의응답 시스템입니다. 여러 문서와 질문을 바탕으로 정확하고 도움이 되는 답변을 제공해야 합니다.

아래 형식으로 여러 문서와 질문이 제공될 것입니다:

질문: {question}

문서 Lists: 
{docs}


다음 지침을 따르세요:

1. 제공된 모든 문서를 주의 깊게 읽고 분석하세요.

2. 주어진 질문을 꼼꼼히 검토하세요.

3. 질문과 가장 관련 있는 문서들을 선별하세요. 질문과 관련 없는 문서는 무시하세요.

4. 관련 문서의 정보를 바탕으로 질문에 대한 명확하고 간결한 답변을 작성하세요.

5. 답변은 반드시 제공된 문서들의 내용에 기반해야 합니다. 문서에 없는 정보로 추측하지 마세요.

6. 여러 문서에서 정보를 종합하여 답변할 수 있습니다. 이 경우, 정보 간의 일관성을 확인하세요.

7. 문서들 간에 상충되는 정보가 있다면, 그 사실을 언급하고 가장 신뢰할 만한 정보를 중심으로 답변하세요.

8. 필요한 경우 특정 문서의 내용을 인용하거나 참조하세요. 이때 어떤 문서에서 정보를 가져왔는지 명시하세요.

9. 모든 문서가 질문과 관련이 없다면, "제공된 문서에서 질문에 대한 관련 정보를 찾을 수 없습니다."라고 답변하세요.

10. 답변 끝에 "추가 질문이 있으시면 언제든 물어보세요."라고 덧붙이세요.

답변 시 항상 공손하고 전문적인 톤을 유지하세요. 이제 주어진 문서들과 질문에 답하실 준비가 되었습니다.
"""
)



In [29]:
from langchain_openai import ChatOpenAI
from typing import List

question = "How to avoid muscle soreness after running?"

def format_docs(docs: List) -> str:
    '''Format the docs.'''
    return "\n\n".join([f"문서{i+1}: {doc['column2']}" for i, doc in enumerate(docs)]) 

llm = ChatOpenAI(model="gpt-4o", temperature=0)


prompt = ANSWER_PROMPT_TEMPLATE.format(
    question=question,
    docs=format_docs(results)
)

response = llm.invoke(prompt)

In [30]:
import pprint
from IPython.display import HTML


def format_content(content):
    # 줄바꿈 처리를 미리 수행합니다.
    formatted_content = content.replace('\n\n', '</p><p>').replace('\n', '<br>')
    return formatted_content

# response 객체가 있다고 가정합니다. 실제 환경에 맞게 이 부분을 조정하세요.
response_content = format_content(response.content)

html_content = f'''
<div style="font-family: Arial, sans-serif; max-width: 800px; margin: auto; line-height: 1.6;">
    <h2 style="color: #333;">AI 응답</h2>
    <div style="border-left: 5px solid #4CAF50; padding: 10px;">
        {response_content}
    </div>
</div>
'''

# HTML 표시
HTML(html_content)